In [1]:
import tensorflow as tf
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
dataset = pd.read_csv("/content/A_Z Handwritten Data.csv").astype('float32')
dataset.rename(columns={'0':'label'}, inplace=True)

# Split into data and labels 
alpha_data = dataset.drop('label',axis = 1)
alpha_labels = dataset['label']

In [3]:
# The below step is to make sure both mnist and the alphabet dataset are of the same dimension.
# The standard mnist dataset has 3 dimensions while alphabet has 2. So a separate dataset was used from
# Kaggle where the dataset was split into train and test. They have been combined in the below code.

dataset = pd.read_csv("/content/mnist_train.csv").astype('float32')
dataset.rename(columns={'0':'label'}, inplace=True)

digit_data1 = dataset.drop('label',axis = 1)
digits_labels1 = dataset['label']

dataset = pd.read_csv("/content/mnist_test.csv").astype('float32')
dataset.rename(columns={'0':'label'}, inplace=True)

digit_data2 = dataset.drop('label',axis = 1)
digits_labels2 = dataset['label']

# append both the datasets together
digit_data = digit_data1.append(digit_data2)
digits_labels = digits_labels1.append(digits_labels2)

In [4]:
# Change the labels to avoid overlapping with the digit dataset
alpha_labels += 10

#combine both the datasets into a single dataset
data = np.vstack([alpha_data, digit_data])
labels = np.hstack([alpha_labels, digits_labels])

#expand the dimensions and normalise the data
data = np.expand_dims(data, axis=-1)
data = data / 255.0

# Splitting the dataset below
(x_train, x_test, y_train, y_test) =  train_test_split(data, labels, test_size=0.25, random_state=42)

In [5]:
# Reshape the data into 3 dimensions. keras.backend is used to adjust teh data based on the channel ordering

import tensorflow.keras.backend as K

if K.image_data_format() == 'channels_first':
  x_train = x_train.reshape(x_train.shape[0], 1, 28,28)
  x_test = x_test.reshape(x_test.shape[0], 1, 28, 28)
  input_shape = (1, 28, 28)
else:
  x_train = x_train.reshape(x_train.shape[0],28, 28, 1)
  x_test = x_test.reshape(x_test.shape[0],28, 28, 1)
  input_shape = (28, 28, 1)

In [6]:
# Image augmentation
aug = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    shear_range=0.2,
    fill_mode='nearest'
) 

In [26]:
# A typical deep learning model with two middle layers.

model = tf.keras.models.Sequential([
    # First two convolutions
    tf.keras.layers.Conv2D(32, (3,3), input_shape=(28, 28, 1), activation='relu'),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Dropout(0.25),
    # Next two convolutions
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPool2D(2, 2),
    tf.keras.layers.Dropout(0.25),
    # Flatten the results
    tf.keras.layers.Flatten(input_shape=(28,28)),
    # Two 256 neuron hidden layers
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(256, activation='relu'),
    # Output layer with 36 classes for all digits and letters
    tf.keras.layers.Dense(36, activation='softmax')
])

opt = tf.keras.optimizers.SGD(lr=0.001)
model.compile(optimizer=opt, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [27]:
import warnings
warnings.filterwarnings('ignore')

In [28]:
model.fit(
    x_train,
    y_train,
    epochs=20
)

Epoch 1/20
10370/10370 [==============================] - 756s 73ms/step - loss: 2.8293 - accuracy: 0.2667
Epoch 2/20
10370/10370 [==============================] - 751s 72ms/step - loss: 0.5945 - accuracy: 0.8264
Epoch 3/20
10370/10370 [==============================] - 753s 73ms/step - loss: 0.3922 - accuracy: 0.8848
Epoch 4/20
10370/10370 [==============================] - 751s 72ms/step - loss: 0.3192 - accuracy: 0.9065
Epoch 5/20
10370/10370 [==============================] - 750s 72ms/step - loss: 0.2762 - accuracy: 0.9184
Epoch 6/20
10370/10370 [==============================] - 746s 72ms/step - loss: 0.2480 - accuracy: 0.9273
Epoch 7/20
10370/10370 [==============================] - 748s 72ms/step - loss: 0.2260 - accuracy: 0.9331
Epoch 8/20
10370/10370 [==============================] - 748s 72ms/step - loss: 0.2102 - accuracy: 0.9380
Epoch 9/20
10370/10370 [==============================] - 748s 72ms/step - loss: 0.2015 - accuracy: 0.9413
Epoch 10/20
10370/10370 [============

In [29]:
loss, accuracy = model.evaluate(x_test, y_test)
print(accuracy*100)
print(loss)

3457/3457 [==============================] - 64s 18ms/step - loss: 0.0990 - accuracy: 0.9715
97.14771509170532
0.09901438653469086


In [23]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
 
# load and prepare the image
def load_image(filename):
	# load the image
	img = load_img(filename, grayscale=True, target_size=(28, 28))
	# convert to array
	img = img_to_array(img)
	# reshape into a single sample with 1 channel
	img = img.reshape(1, 28, 28, 1)
	# prepare pixel data
	img = img.astype('float32')
	img = img / 255.0
	return img
 
# load an image and predict the class
def run_example():
	# load the image
	img = load_image('7.png')
	# predict the class
	digit = model.predict_classes(img)
	print(digit[0])
 
# entry point, run the example
run_example()

7
